## Step 1 from Assignment
1. Load the global weather data into your big data technology of choice.
2. Join the stationlist.csv with the countrylist.csv to get the full country name
for each station number.
3. Join the global weather data with the full country names by station number.

In [1]:
#import pyspark spark session
from pyspark.sql import SparkSession

#create spark session
#set app name to the assigment challenge
spark = SparkSession.\
        builder.\
        appName("de-weather-challenge").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

In [8]:
# step 1.1 load data
station_df = spark.read.csv(path="../../data/stationlist.csv", sep=",", header=True)
station_df.count()

25306

In [3]:
station_df.show(10)

+------+------------+
|STN_NO|COUNTRY_ABBR|
+------+------------+
|012240|          NO|
|020690|          SW|
|020870|          SW|
|021190|          SW|
|032690|          UK|
|033450|          UK|
|039290|          UK|
|039790|          EI|
|040480|          IC|
|041300|          IC|
+------+------------+
only showing top 10 rows



In [4]:
# get distinct country abbreviations
station_df.select("COUNTRY_ABBR").distinct().count()

251

In [5]:
country_df = spark.read.csv(path="../../data/countrylist.csv", sep=",", header=True)
country_df.show(10)

+------------+-------------------+
|COUNTRY_ABBR|       COUNTRY_FULL|
+------------+-------------------+
|          AA|              ARUBA|
|          AC|ANTIGUA AND BARBUDA|
|          AF|        AFGHANISTAN|
|          AG|            ALGERIA|
|          AI|   ASCENSION ISLAND|
|          AJ|         AZERBAIJAN|
|          AL|            ALBANIA|
|          AM|            ARMENIA|
|          AN|            ANDORRA|
|          AO|             ANGOLA|
+------------+-------------------+
only showing top 10 rows



In [6]:
country_df.count()

288

Finding: there are 288 countries in the countrylist.csv but only 251 distinct country abbreviations in stationlist.csv so there are missing countries

Assumption: I will use an inner join to join country_df and station_df since all the questions require weather data from stations. If a country does not have a station, then I am assuming it does not have any weather data.

In [7]:
weather_df = spark.read.csv(path="../../data/data/*", sep=",", header=True)
weather_df.show()

+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+
|STN---| WBAN|YEARMODA|TEMP|DEWP|   SLP|   STP|VISIB|WDSP|MXSPD| GUST|  MAX|  MIN| PRCP| SNDP|FRSHTT|
+------+-----+--------+----+----+------+------+-----+----+-----+-----+-----+-----+-----+-----+------+
|010260|99999|20190101|26.1|21.2|1001.9| 987.5| 20.6| 9.0| 15.9| 29.7| 29.8|21.7*|0.02G| 18.5|001000|
|010260|99999|20190102|24.9|22.1|1020.1|1005.5|  5.4| 5.6| 13.6| 22.1|27.1*| 20.7|0.48G| 22.8|001000|
|010260|99999|20190103|31.7|29.1|1008.9| 994.7| 13.6|11.6| 21.4| 49.5|37.4*|26.8*|0.25G|999.9|011000|
|010260|99999|20190104|32.9|30.3|1011.4| 997.1| 15.8| 4.9|  7.8| 10.9| 36.1| 31.8|0.52G|999.9|001000|
|010260|99999|20190105|35.5|33.0|1015.7|1001.4| 12.0|10.4| 13.6| 21.0|38.5*| 32.7|0.02G| 23.6|010000|
|010260|99999|20190106|38.5|34.1|1008.2| 994.2| 12.8|10.0| 17.5| 28.9| 41.4|33.8*|0.12G| 23.2|010000|
|010260|99999|20190107|32.1|29.8| 996.8| 982.7|  6.9|11.3| 15.5| 28.6|35.1*| 30.4|

In [ ]:
#step 1.2 join country and station list